In [1]:
#import required libraries
import pandas as pd
import requests
import numpy as np

In [2]:
#Task1: All alphabetical characters in all columns should be capitalized except (DOB, POBox, POCityStateZip).
data = pd.read_csv('inputDB.csv')

for i in data.columns: 
    if i not in ['DOB','POBox','POCityStateZip']:
        data[i] = data[i].str.upper()

In [3]:
#Task2: Address data should be compatible to the standard

data = pd.read_csv('inputDB.csv')

length = len(data)
is_address = np.zeros(length, dtype='str')

is_address = data['Address'].str.split(' ')
is_address = is_address.apply(lambda x:True if (x[0].isdigit() and len(x[0])>=3 and len(x[0]) <=5)  else False)
is_address = pd.DataFrame(is_address)

zip_addr = list(zip(data['Address'],is_address['Address']))
zip_addr = pd.Series(zip_addr)
data['Address'] = zip_addr.apply(lambda x: str(x[0]).upper() if x[1] == True else str(x[0]).upper())

In [26]:
#Task3: The State column should contain the correct two character US state code.

#Create a dictionary for states {'two letter names' : 'full names'}
states = {
    'AK': 'Alaska',
    'AL': 'Alabama',
    'AR': 'Arkansas',
    'AS': 'American Samoa',
    'AZ': 'Arizona',
    'CA': 'California',
    'CO': 'Colorado',
    'CT': 'Connecticut',
    'DC': 'District of Columbia',
    'DE': 'Delaware',
    'FL': 'Florida',
    'GA': 'Georgia',
    'GU': 'Guam',
    'HI': 'Hawaii',
    'IA': 'Iowa',
    'ID': 'Idaho',
    'IL': 'Illinois',
    'IN': 'Indiana',
    'KS': 'Kansas',
    'KY': 'Kentucky',
    'LA': 'Louisiana',
    'MA': 'Massachusetts',
    'MD': 'Maryland',
    'ME': 'Maine',
    'MI': 'Michigan',
    'MN': 'Minnesota',
    'MO': 'Missouri',
    'MP': 'Northern Mariana Islands',
    'MS': 'Mississippi',
    'MT': 'Montana',
    'NA': 'National',
    'NC': 'North Carolina',
    'ND': 'North Dakota',
    'NE': 'Nebraska',
    'NH': 'New Hampshire',
    'NJ': 'New Jersey',
    'NM': 'New Mexico',
    'NV': 'Nevada',
    'NY': 'New York',
    'OH': 'Ohio',
    'OK': 'Oklahoma',
    'OR': 'Oregon',
    'PA': 'Pennsylvania',
    'PR': 'Puerto Rico',
    'RI': 'Rhode Island',
    'SC': 'South Carolina',
    'SD': 'South Dakota',
    'TN': 'Tennessee',
    'TX': 'Texas',
    'UT': 'Utah',
    'VA': 'Virginia',
    'VI': 'Virgin Islands',
    'VT': 'Vermont',
    'WA': 'Washington',
    'WI': 'Wisconsin',
    'WV': 'West Virginia',
    'WY': 'Wyoming'
}

data = pd.read_csv('inputDB.csv')

#-----------Split the dictionary for key, values and full items----------
states_key = states.keys()
states_value = list(states.values())
states_value = [x.upper() for x in states_value] #convert values to UpperCase
states_item = states.items()
length = len(data)
#------------------------------------------------------------------------

#-----------Declare arrays for each column that we are gonna check the State value in---------
state_from_state = np.zeros(length,dtype='str')
state_from_POCityStateZip = np.zeros(length,dtype='str')
state_from_zip = np.zeros(length,dtype='str')
#--------------------------------------------------------------------

#-----------looking for State from State column itself--------------
c1 = data['State'].isin(states_key)
c2 = data['State'].isin(states_value)

state_from_state = list(zip(data['State'],c2,c1))
state_from_state = pd.Series(state_from_state).apply(lambda z :[shortcut for shortcut, state_name in states_item if str(state_name).upper() == z[0]][0] if z[1] else z[0] if z[2] else '') 
state_from_state = pd.DataFrame(state_from_state)
#--------------------------------------------------------------------

#-----------Looking for State from ZIP column------------------------
state_from_zip = data['ZIP'].str.split(' ')
state_from_zip = state_from_zip.apply(lambda x: x[0] if len(x)==2 else x[1] if len(x)==3 else '')
c1 = state_from_zip.isin(states_key)
c2 = state_from_zip.isin(states_value)
state_from_zip = list(zip(state_from_zip,c2,c1))
state_from_zip = pd.Series(state_from_zip).apply(lambda z :[shortcut for shortcut, state_name in states_item if str(state_name).upper() == z[0]][0] if z[1] else z[0] if z[2] else '') 
state_from_zip = pd.DataFrame(state_from_zip)
#--------------------------------------------------------------------

#------------Looking for State from POCityStateZip column------------
state_from_POCityStateZip = data['POCityStateZip'].str.split(',')
state_from_POCityStateZip = state_from_POCityStateZip.apply(lambda x: x[-1] if str(x)!='nan' else '' )
state_from_POCityStateZip = state_from_POCityStateZip.apply(lambda x: x.split(' ')[-(len(x.split(' '))-1)]if str(x)!='nan' else '' )
c1 = state_from_POCityStateZip.isin(states_key)
c2 = state_from_POCityStateZip.isin(states_value)
state_from_POCityStateZip = list(zip(state_from_POCityStateZip,c2,c1))
state_from_POCityStateZip = pd.Series(state_from_POCityStateZip).apply(lambda z :[shortcut for shortcut, state_name in states_item if str(state_name).upper() == z[0]][0] if z[1] else z[0] if z[2] else '') 
state_from_POCityStateZip = pd.DataFrame(state_from_POCityStateZip)
#--------------------------------------------------------------------

#combine the true values with zip function and edit the State column in the original dataframe
combinedStates = list(zip(state_from_state[0],state_from_POCityStateZip[0],state_from_zip[0]))
combinedStates = pd.DataFrame(combinedStates)
combinedStates
data['State'] = combinedStates.apply(lambda x: x[0] if str(x[0])!='' else x[1] if str(x[1])!= '' else str(x[2]),axis=1)


0          
1          
2          
3        AR
4        CA
5        CA
6        CA
7          
8        FL
9          
10       TX
11         
12         
13       CA
14         
15         
16       CA
17       FL
18         
19       CA
20         
21       AR
22         
23         
24       AR
25         
26       CA
27       AR
28       CA
29         
         ..
94276      
94277      
94278      
94279    FL
94280    FL
94281      
94282    TX
94283      
94284      
94285    FL
94286      
94287    FL
94288      
94289    CA
94290    FL
94291      
94292    TX
94293      
94294    FL
94295      
94296      
94297      
94298    TX
94299    TX
94300      
94301    CA
94302      
94303      
94304      
94305    FL
Name: State, Length: 94306, dtype: object

In [41]:
#Task4: City column should contain real city names.
data = pd.read_csv('inputDB.csv')

length = len(data)
city_from_city = np.zeros(length, dtype='str')

for i in data.columns: 
    if i  in ['City']:#,'POBox','POCityStateZip']:
        data[i] = data[i].str.upper()

is_city = data['City'].notnull()
city_from_city = np.where(is_city, data['City'], city_from_city)

city_from_POCityStateZip = np.zeros(length, dtype='str')
city_from_POCityStateZip = data['POCityStateZip'].str.split(',')
city_from_POCityStateZip = city_from_POCityStateZip.apply(lambda x: x[0] if str(x) != 'nan' else '')

city_from_city = pd.DataFrame(city_from_city)
city_from_POCityStateZip = pd.DataFrame(city_from_POCityStateZip)
city_from_POCityStateZip
true_city = zip(city_from_city[0],city_from_POCityStateZip['POCityStateZip'])
true_city = pd.DataFrame(true_city)
data['City'] = true_city.apply(lambda x: str(x[0]).upper() if x[0]!='' else str(x[1]).upper() ,axis=1)


TypeError: data argument can't be an iterator